In [1]:
import json
import xlwings as xw
import pandas as pd

In [2]:
def addPrices(r: dict) -> dict:
    d = r.copy()
    for k,v in r.items():
        if k in ['SPREAD','RATE','FXPOINTS','FXSPOT']:
            d[k] = prices_df[prices_df.TICKER==v].VALUE.iloc[0]
            d[k+'TICKER'] =v
    return d
def dropNones(d: dict) -> dict:
    r = d.copy()    
    for k, v in d.items():
        if v is None:
            del r[k]
    return r
def exportJson(curve_name, data):
    json_object  = json.dumps(data)
    with open(curve_name+".json", "w") as outfile:
        outfile.write(json_object)

In [3]:
dfs = {}
date_handler = lambda year, month, day, **kwargs: "%02i%02i%04i" % (day, month,year)
for s in xw.books[0].sheets:
    dfs[s.name] = s.range('A1').options(pd.DataFrame, expand='table', index=False, dates=date_handler).value    

In [4]:
curves = {}
exclude = ['CURVES','INDEX','PRICES','XCCYBASIS']
prices_df = dfs['PRICES']
curves_dfs = dfs['CURVES']
for curve in curves_dfs.NAME:
    curve_config = curves_dfs[curves_dfs.NAME==curve].iloc[0].to_dict()
    rate_helpers = []
    for s in xw.books[0].sheets:        
        if s.name not in exclude:
            tmp = dfs[s.name]            
            for r in tmp[tmp.NAME==curve].T.to_dict().values():
                d = dropNones(r)
                d['TYPE'] = s.name
                d = addPrices(d)
                del d['NAME']
                rate_helpers.append(d)
            curve_config['RATEHELPERS'] = rate_helpers
    curves[curve] = curve_config

In [5]:
for k,v in curves.items():
    exportJson(k,v)

In [6]:
indices = {}
indices_dfs = dfs['INDEX']
for index in indices_dfs.NAME:
    index_config = indices_dfs[indices_dfs.NAME==index].iloc[0].to_dict()
    indices[index] = index_config

In [7]:
for k,v in indices.items():
    exportJson('INDEX_'+k,v)